In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Configuración visual
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

# Cargar archivo
df = pd.read_excel(r"C:\Users\iazuaz\PyCharmMiscProject\model_RRSS\analysis\results_mar2025_a_jul2025_v2.xlxs")

# Asegurar tipo datetime
df["Fecha"] = pd.to_datetime(df["Fecha"])

# Filtrar hasta el 30 de abril de 2025
fecha_corte = pd.to_datetime("2025-04-30")
df = df[df["Fecha"] <= fecha_corte].copy()

# Modelos y columna real
modelos = ["Modelo_Antiguo", "Modelo_Nuevo", "Modelo_Nuevo_v2", "Modelo_Gen"]
col_real = "Reales"

# Función MAPE y SMAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.where(y_true == 0, 1e-9, y_true))) * 100

def symmetric_mape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true) + 1e-9))

# Crear DataFrame de métricas
metricas = []

for modelo in modelos:
    y_true = df[col_real]
    y_pred = df[modelo]

    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    smape = symmetric_mape(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    metricas.append({
        "Modelo": modelo,
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "MAPE (%)": mape,
        "SMAPE (%)": smape,
        "R²": r2
    })

df_metricas = pd.DataFrame(metricas)

# Identificar mejor modelo por cada métrica
mejores = df_metricas.drop(columns=["Modelo"]).idxmin()
mejores["R²"] = df_metricas["R²"].idxmax()  # Para R², el mejor es el más alto

print("📊 Métricas de rendimiento hasta el 30 de abril de 2025:")
display(df_metricas.style.highlight_min(subset=["MAE", "MSE", "RMSE", "MAPE (%)", "SMAPE (%)"], color='lightgreen')
                         .highlight_max(subset=["R²"], color='lightgreen'))

print("\n🏆 Mejor modelo por métrica:")
print(mejores)

# Visualización de métricas
df_metricas_melt = df_metricas.melt(id_vars="Modelo", var_name="Métrica", value_name="Valor")

plt.figure(figsize=(14, 6))
sns.barplot(data=df_metricas_melt, x="Modelo", y="Valor", hue="Métrica")
plt.title("Comparación de métricas por modelo")
plt.ylabel("Valor")
plt.legend(title="Métrica")
plt.tight_layout()
plt.show()

# Cálculo de errores absolutos diarios
for modelo in modelos:
    df[f"Error_Abs_{modelo}"] = np.abs(df[col_real] - df[modelo])

# Promedio de error absoluto diario
df_errores_diarios = df[["Fecha"] + [f"Error_Abs_{m}" for m in modelos]].copy().set_index("Fecha")

# Visualización de errores absolutos diarios
plt.figure(figsize=(14, 6))
for modelo in modelos:
    plt.plot(df_errores_diarios.index, df_errores_diarios[f"Error_Abs_{modelo}"], label=modelo)

plt.title("Error absoluto diario por modelo")
plt.xlabel("Fecha")
plt.ylabel("Error absoluto")
plt.legend()
plt.tight_layout()
plt.show()

# Error absoluto promedio por día del mes
df["Día_Mes"] = df["Fecha"].dt.day
errores_dia_mes = df.groupby("Día_Mes")[[f"Error_Abs_{m}" for m in modelos]].mean().reset_index()

plt.figure(figsize=(12, 6))
for modelo in modelos:
    plt.plot(errores_dia_mes["Día_Mes"], errores_dia_mes[f"Error_Abs_{modelo}"], marker='o', label=modelo)

plt.title("Error absoluto promedio por día del mes")
plt.xlabel("Día del mes")
plt.ylabel("Error absoluto promedio")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Error absoluto promedio por día de la semana
dias_semana = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]
df["Día_Semana"] = df["Fecha"].dt.dayofweek
df["Nombre_Día_Semana"] = df["Día_Semana"].apply(lambda x: dias_semana[x])

errores_semana = df.groupby("Nombre_Día_Semana")[[f"Error_Abs_{m}" for m in modelos]].mean()
errores_semana = errores_semana.reindex(dias_semana)

errores_semana.plot(kind="bar", figsize=(12, 6))
plt.title("Error absoluto promedio por día de la semana")
plt.ylabel("Error absoluto promedio")
plt.xlabel("Día de la semana")
plt.grid(axis="y")
plt.tight_layout()
plt.show()

# Comparación visual: Predicciones vs Reales
plt.figure(figsize=(14, 6))
plt.plot(df["Fecha"], df["Reales"], label="Reales", linewidth=2, color="black")
for modelo in modelos:
    plt.plot(df["Fecha"], df[modelo], label=modelo, alpha=0.8)

plt.title("Comparación de valores Reales vs Predicciones")
plt.xlabel("Fecha")
plt.ylabel("Consultas recibidas")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
